In [ ]:
import requests                 # 모듈
from bs4 import BeautifulSoup   # 클래스 

# url = 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/' # 메인페이지
url_main = url_base + url_sub   


headers = {
    'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36'
}
response = requests.get(url=url_main,headers=headers)
# print(response.text[:1000])

#파싱 >> 파싱트리(통칭 DOM tree)생성
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
tmp = soup.find('div', class_='sammy')
# print(tmp)
tmp.find(class_='sammyRank').text

In [ ]:
tmp.find(class_='sammyListing')
tmp.find(class_='sammyListing').text
tmp.find(class_='sammyListing').text.split('\n')[:2]

In [ ]:
url_a = tmp.find('a').get('href') # 근데 50개중에 https 로 시작하는 링크도 있음 ㅋㅋ 이걸로 다 안 됨
print(url_base+url_a)

In [ ]:
# 메인에서 50개 데이터 접근, 추출
import requests
from bs4 import BeautifulSoup
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/' # 메인페이지
url_main = url_base + url_sub   


headers = {
    'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36'
}
response = requests.get(url=url_main,headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# 메인에서 데이터 추출: 순위, 메인메뉴, 카페명, 상세URL
ranks=[]
main_menus=[]
cafe_menus=[]
url_adds = []

list_sammy=soup.find_all('div', class_='sammy')
# print(len(list_sammy))
for item in list_sammy:
    ranks.append(item.find(class_='sammyRank').text)
    tmp_str = item.find(class_='sammyListing').text.split('\n')[:2]
    main_menus.append(tmp_str[0])
    cafe_menus.append(tmp_str[1])
    
    
    tmp_url = item.find('a')['href']
    if tmp_url.startswith('https://'):
        url_adds.append(tmp_url)
    else:
        url_adds.append(url_base + tmp_url)

In [ ]:
print(ranks[:10])
print(main_menus[:10])
print(cafe_menus[:10])
print(url_adds[:10])

In [ ]:
import pandas as pd
import numpy as np
data = pd.DataFrame({
    'Rank':ranks,
    'Menu':main_menus,
    'Cafe':cafe_menus,
    'Url':url_adds
})

In [ ]:
# 1등 페이지에 가서 가격,전체 주소 추출
url_str = data['Url'][0]
headers = {
    'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36'
}
response = requests.get(url=url_str,headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
tmp_str=soup.find('p', class_='addy').text
# tmp_str.split()[0]                  # 가격 추출인데 . 붙어있음 ㅠ 
# tmp_str.split()[0][:-1]             # 가격 추출
tmp_str.split()[1:-2]               
' '.join(tmp_str.split()[1:-2])       # 주소 추출

In [ ]:
import time
from tqdm import tqdm
for number in tqdm(range(10)): # 오 신기하당
    time.sleep(1)

In [ ]:
price = []
address = []
for idx in tqdm(data.index):
    url_str = data['Url'][idx]
    headers = {
    'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36'
    }
    response = requests.get(url=url_str,headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    get_str = soup.find('p', class_='addy').text
    price.append(get_str.split()[0][:-1])
    address.append(' '.join(get_str.split()[1:-2]))
    

In [ ]:
print(price[11],address[11])        # 주소중에 Multiple?? << 지점이 여러개인거임 ㅋㅋ

In [ ]:
data['Price']=price
data['Address']=address
data.head()

In [ ]:
data.set_index('Rank', inplace=True)
data.head()

In [ ]:
# 자동로그인 : 수동으로 로그인된 브라우저의 사용자프로필로 저리 가능 (쿠키정보)
# 근데 본인 쿠키 이용할 줄 모름 ㅋㅋ 약관위반 사항이니 주의